In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("restaurant-review-average") \
    .master("local[*]") \
    .getOrCreate()

In [2]:
sc = spark.sparkContext  # 필요 시 RDD API를 위해 사용

In [3]:
sc.defaultParallelism

2

In [4]:
sc.setLogLevel('INFO')

In [5]:
sc.getConf().getAll()

[('spark.app.id', 'local-1754354311144'),
 ('spark.app.name', 'restaurant-review-average'),
 ('spark.driver.host', 'b81dfefa0d89'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHand

In [6]:
data = [
    (0, "짜장면", "중식", 125),
    (1, "짬뽕", "중식", 235),
    (2, "김밥", "분식", 32),
    (3, "떡볶이", "분식", 534),
    (4, "라멘", "일식", 223),
    (5, "돈가스", "일식", 52),
    (6, "우동", "일식", 12),
    (7, "쌀국수", "아시안", 312),
    (8, "햄버거", "패스트푸드", 12),
    (9, "치킨", "패스트푸드", 23),
]

In [7]:
rdd1 = sc.parallelize(data)

In [8]:
rdd1.take(3)

[(0, '짜장면', '중식', 125), (1, '짬뽕', '중식', 235), (2, '김밥', '분식', 32)]

In [9]:
lines = sc.textFile("file:///home/jovyan/work/learning_spark_data/restaurant_reviews.csv")
lines.take(5)

Py4JJavaError: An error occurred while calling o90.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/jovyan/work/learning_spark_data/restaurant_reviews.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.api.java.JavaRDDLike.partitio

In [ ]:
#총건수 확인
# 첫줄, 데이터 분할
header = lines.first()
filtered_lines = lines.filter(lambda row:row!=header)
# map으로 split

In [ ]:
sc.uiWebUrl

In [ ]:
sc.stop()
spark.stop()

In [ ]:
def parse(row):
    fields = row.split(",")
    
    category = fields[2] #카테고리
    
    # reviews는 정수로 parse
    reviews = fields[3] # 리뷰수
    reviews = int(reviews)
    
    return category, reviews

In [ ]:
parse('0,짜장면,중식,125,')

In [ ]:
category_reviews = filtered_lines.map(parse)
category_reviews

In [ ]:
category_reviews.collect()

In [ ]:
category_review_count = category_reviews.mapValues(lambda x : (x, 1)) # x는 review 개수
category_review_count.collect()

In [ ]:
#같은 key(카테고리)끼리 x와 y의 (리뷰 수 합계, 개수 합계)를 누적
#첫번째 중식 x, 두번째 중식 y , [0] 은 리뷰수, [1] 은 개수 -> 같은 키끼리 행을 바꿔가며 계속 누적해 나간다.
reduced = category_review_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduced.collect()

In [ ]:
average = reduced.mapValues(lambda x : x[0] / x[1])
average.collect()

In [ ]:
# Persist 를 사용하지 않을 때

In [ ]:
# header = lines.first()
# filtered_lines = lines.filter(lambda row : row != header)
filtered_lines.collect()

In [ ]:
# transformations를 수행할 RDD 생성
categoryReviews = filtered_lines.map(parse)
categoryReviews.collect()

In [ ]:
result1 = categoryReviews.take(10) # action을 곧바로 실행

In [ ]:
result2 = categoryReviews.mapValues(lambda x : (x, 1)).collect()

In [ ]:
# persist를 사용하는 경우

In [ ]:
#job x
categoryReviews = filtered_lines.map(parse).persist() # categoryReviews RDD는 하나만 존재하는 RDD
categoryReviews

In [ ]:
result3 = categoryReviews.take(10)
result3

In [ ]:
result4 = categoryReviews.mapValues(lambda x : (x, 1)).collect()
result4

In [ ]:
# 카테고리별 값1 합계 계산 (첫 번째 연산)
result_sum = categoryReviews \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
result_sum

In [ ]:
# (카테고리, (점수, 1)) 형태로 변환
rdd_with_count = categoryReviews.map(lambda x: (x[0], (x[1], 1)))
rdd_with_count.take(3)

In [ ]:
# reduceByKey로 (점수 총합, 개수 총합) 집계
rdd_sums = rdd_with_count.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
rdd_sums.take(3)

In [ ]:
# 평균 계산
rdd_avg = rdd_sums.mapValues(lambda x: round(x[0] / x[1], 2))

# 결과 출력
rdd_avg.collect()

In [ ]:
# 기존 저장 레벨 해제
categoryReviews.unpersist()

In [ ]:
from pyspark import StorageLevel
categoryReviews.persist(StorageLevel.MEMORY_AND_DISK)

In [ ]:
# 구조 확인하기
categoryReviews.take(3)

In [ ]:
result11 = categoryReviews.filter(lambda x: x[1] > 100).count()
result12 = categoryReviews.filter(lambda x: x[1] <= 100).count()

# Narrow Transformations
1:1 변환 -> 하나의 열을 다룰 때 다른 데이터가 필요 없는 경우
filter(), map(), flatMap(), sample(), union()

# wide transformations

# flatMap()

In [ ]:
rdd = sc.parallelize([1, 2, 3])
rdd_map = rdd.map(lambda x: [x, x + 1])  # => [[1, 2], [2, 3], [3, 4]]
rdd_map.collect()

In [ ]:
rdd_flatmap = rdd.flatMap(lambda x: [x, x + 1])  # => [1, 2, 2, 3, 3, 4]
rdd_flatmap.collect()

In [ ]:
# 텍스트를 단어로 쪼개서 flatMap()

In [ ]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [ ]:
moviesRDD = sc.parallelize(movies)
moviesRDD

In [ ]:
#job 추가 x
flatMovies = moviesRDD.flatMap(lambda x : x.split(" "))
flatMovies.collect()

In [ ]:
# 집합 Transformation

In [ ]:
num1 = sc.parallelize([1, 2, 3, 4, 5])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [ ]:
num1.intersection(num2).collect()

In [ ]:
# 합집합 구하기 - union

In [ ]:
num_union = num1.union(num2)
num_union

In [ ]:
num_union.collect()

In [ ]:
# 차집합 구하기 - subtract

In [ ]:
num1.subtract(num2).collect()

In [ ]:
# 데이터 랜덤 추출 - sample(withReplacement, fraction, seed=None)

In [ ]:
# withReplacement : True -> 중복 추출
numUnion.sample(True, 0.3).collect()

In [ ]:
# withReplacement : False -> 중복 X
numUnion.sample(False, 0.7).collect()

In [ ]:
# 랜덤을 고정해서 항상 같은 결과가 나올 수 있도록
numUnion.sample(True, 0.5, seed=42).collect()

In [ ]:
# Wide Transformations

In [ ]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

In [ ]:
# 그룹핑의 기준을 문자열의 첫 번째 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup

In [ ]:
res = foodsGroup.collect()

In [ ]:
for (k, v) in res:
    print(k, list(v))

In [ ]:
# 내로우 narrow  트랜스포메이션

In [ ]:
### 2. Narrow 트랜스포메이션 예제
numbers = sc.parallelize([1, 2, 3, 4, 5])

# 1:1 변환 (Narrow Transformation)
sample_rdd = numbers.sample(False, 0.5)
sample_rdd.take(3)

In [ ]:
union_rdd = numbers.union(sc.parallelize([6,7,8]))
union_rdd.collect()

In [ ]:
#wide 연산
kv_rdd = sc.parallelize([("apple", 1), ("banana", 2), ("cherry", 3)])

In [ ]:
mapp_values = kv_rdd.mapValues(lambda x: x*10)
mapp_values.collect()

In [ ]:
#wide 연산
grouped_rdd = kv_rdd.groupByKey()
grouped_rdd.collect()

In [ ]:

reduced_rdd = kv_rdd.reduceByKey(lambda x,y: x+y)
reduced_rdd.collect()

In [ ]:
# 조인 연산

In [ ]:
sc.stop()

In [ ]:
spark.stop()